In [1]:
#!/usr/bin/env python3
"""
Monte‑Carlo vs 解析解（対数正規）比較バッチ
  – 100 個の乱数シードを回し，
    ・生成した正規乱数（月次）を CSV 保存
    ・パス数 5 000 / 10 000 で Monte‑Carlo を 2 通り実行
    ・分位点誤差・KS 距離・JS 距離を計算
    ・結果を 1 本の CSV に集約
────────────────────────────────────────────
Python ≥3.9 ・NumPy ・SciPy ・pandas があれば動作します
"""

import os
import numpy as np
import pandas as pd
from pathlib import Path
from scipy.stats import norm, kstest
from scipy.spatial.distance import jensenshannon

# ──────────────────────────────────────────
# 1. モデル・シミュレーション関数
# ──────────────────────────────────────────
def rs_to_mu_sigma(r: float, s: float) -> tuple[float, float]:
    """算術平均 r, 標準偏差 s → 連続複利 μ, σ（厳密変換）"""
    sigma2 = np.log(1.0 + (s / (1.0 + r)) ** 2)
    mu = np.log(1.0 + r) - 0.5 * sigma2
    return mu, np.sqrt(sigma2)


def analytic_quantiles(mu_sde: float, sigma: float,
                       S0: float, T: int,
                       qs: np.ndarray) -> np.ndarray:
    """GBM の閉形式分位点（μ_sde = 価格ドリフト）"""
    z = norm.ppf(qs)
    return S0 * np.exp((mu_sde - 0.5 * sigma ** 2) * T +
                       sigma * np.sqrt(T) * z)


def model_pdf(x: np.ndarray, mu_sde: float, sigma: float,
              S0: float, T: int) -> np.ndarray:
    """対数正規 PDF（GBM 終値）"""
    log_term = (np.log(x / S0) -
                (mu_sde - 0.5 * sigma ** 2) * T)
    return (1.0 / (x * sigma * np.sqrt(2.0 * np.pi * T))
            * np.exp(-log_term ** 2 / (2.0 * sigma ** 2 * T)))


def gbm_monthly_paths(Y: np.ndarray,
                      mu_log: float, sigma: float,
                      S0: float, mon: int) -> np.ndarray:
    """
    月次ホワイトノイズ Y から価格パスを生成（ベクトル化版）
    Y.shape = (Npaths, months)
    """
    mu_m = mu_log / mon
    sigma_m = sigma / np.sqrt(mon)
    R_month = np.exp(mu_m + sigma_m * Y) - 1.0
    return S0 * np.prod(1.0 + R_month, axis=1)


# ──────────────────────────────────────────
# 2. 入力パラメータ
# ──────────────────────────────────────────
r       = 0.077    # 算術平均リターン
s       = 0.111     # 算術ボラティリティ
S0      = 700.0     # 初期元本
T       = 30        # 投資期間（年）
T_rand  = 100       # 乱数を多めに確保しておく年数
mon     = 12        # 1 年 = 12 か月
N1      = 5_000     # Monte‑Carlo パス (その 1)
N2      = 10_000    # Monte‑Carロ パス (その 2)
QS      = np.array([0.10, 0.30, 0.50, 0.70, 0.90])  # 分位点

# 出力先フォルダ
out_dir = Path("mc_batch_output")
out_dir.mkdir(exist_ok=True)

# ──────────────────────────────────────────
# 3. 事前計算
# ──────────────────────────────────────────
mu_log, sigma = rs_to_mu_sigma(r, s)
mu_sde = mu_log + 0.5 * sigma ** 2        # 価格ドリフト
ana_q  = analytic_quantiles(mu_sde, sigma, S0, T, QS)

# ──────────────────────────────────────────
# 4. ループ本体
# ──────────────────────────────────────────
records = []       # 結果をここに溜める

for seed in range(100):
    rng = np.random.default_rng(seed)

    # 乱数生成（N2 パスぶんまとめて確保、必要分だけ切り出す）
    Y_all = rng.normal(loc=0.0, scale=1.0,
                       size=(N2, T_rand * mon))

    # 乱数 CSV 保存（列 = パス、行 = 月次）
    rand_file = out_dir / f"rand_{seed:02d}.csv"
    # np.savetxt(rand_file, Y_all.T, delimiter=",")

    # Monte‑Carlo を 2 つのパス数で評価
    for n_paths in (N1, N2):
        Y_use = Y_all[:n_paths, :T * mon]
        prices = gbm_monthly_paths(Y_use, mu_log, sigma, S0, mon)

        # 分位点差
        mc_q = np.quantile(prices, QS)
        diff = 100*(mc_q - ana_q)/ana_q

        # KS 距離（解析 CDF に対する適合度）
        analytic_cdf = lambda x: norm.cdf(
            (np.log(x / S0) - (mu_sde - 0.5 * sigma ** 2) * T)
            / (sigma * np.sqrt(T)))
        ks_stat, _ = kstest(prices, analytic_cdf)

        # JS 距離（ヒスト＋解析 PDF）
        bins = np.linspace(prices.min(), prices.max(), 400)
        hist_mc, _ = np.histogram(prices, bins=bins, density=True)
        centers = 0.5 * (bins[1:] + bins[:-1])
        pdf_ana  = model_pdf(centers, mu_sde, sigma, S0, T)
        # Avoid zero‑division
        hist_mc += 1e-12
        pdf_ana  += 1e-12
        js_div = jensenshannon(hist_mc, pdf_ana) ** 2  # square → JS divergence

        # レコード作成
        rec = {
            "seed":      seed,
            "paths":     n_paths,
            "KS":        ks_stat,
            "JS":        js_div,
        }
        # 分位点差を列追加
        for q, d in zip(QS, diff):
            rec[f"diff_{int(q*100)}"] = d
        records.append(rec)

    print(f"seed {seed:02d} … done")

# ──────────────────────────────────────────
# 5. CSV 出力
# ──────────────────────────────────────────
df = pd.DataFrame(records)
cmp_file = out_dir / "mc_vs_analytic_summary.csv"
df.to_csv(cmp_file, index=False)

print("\n==== バッチ完了 ====")
print(f"乱数 CSV: {out_dir}/rand_XX.csv  (XX = 00–99)")
print(f"比較結果  : {cmp_file}")


seed 00 … done
seed 01 … done
seed 02 … done
seed 03 … done
seed 04 … done
seed 05 … done
seed 06 … done
seed 07 … done
seed 08 … done
seed 09 … done
seed 10 … done
seed 11 … done
seed 12 … done
seed 13 … done
seed 14 … done
seed 15 … done
seed 16 … done
seed 17 … done
seed 18 … done
seed 19 … done
seed 20 … done
seed 21 … done
seed 22 … done
seed 23 … done
seed 24 … done
seed 25 … done
seed 26 … done
seed 27 … done
seed 28 … done
seed 29 … done
seed 30 … done
seed 31 … done
seed 32 … done
seed 33 … done
seed 34 … done
seed 35 … done
seed 36 … done
seed 37 … done
seed 38 … done
seed 39 … done
seed 40 … done
seed 41 … done
seed 42 … done
seed 43 … done
seed 44 … done
seed 45 … done
seed 46 … done
seed 47 … done
seed 48 … done
seed 49 … done
seed 50 … done
seed 51 … done
seed 52 … done
seed 53 … done
seed 54 … done
seed 55 … done
seed 56 … done
seed 57 … done
seed 58 … done
seed 59 … done
seed 60 … done
seed 61 … done
seed 62 … done
seed 63 … done
seed 64 … done
seed 65 … done
seed 66 … 

In [5]:
print(ana_q)

[ 2687.68969421  4116.35926221  5530.14624925  7429.5064133
 11378.73825389]


In [4]:
# ベスト乱数
rng = np.random.default_rng(57)
Y_all = rng.normal(loc=0.0, scale=1.0,size=(N2, T_rand * mon))
np.savetxt('best_rand.csv', Y_all.T, delimiter=",",fmt="%.8f",newline="\r\n")